In [3]:
import pandas as pd
from pulse import *

df = pd.read_csv('labeled_data_prep.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,label,tweet,clean_tweet,protected_attribute
0,0,1,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,woman complain cleaning house amp man always t...,0
1,1,1,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,boy dats cold tyga dwn bad cuffin dat hoe st p...,0
2,2,1,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,dawg ever fuck bitch sta cry confused shit,0
3,3,1,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,g anderson based look like tranny,0
4,4,1,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,shit hear might true might faker bitch told ya,0


In [4]:
overall_auc, subgroup_auc, bpsn_auc, bnsp_auc, privileged_group_acc, subgroup_accuracy, acc_delta = detect_bias_subgroup(df, 'clean_tweet', 'protected_attribute', multi_class=True)

              precision    recall  f1-score   support

           0       0.51      0.13      0.21       287
           1       0.92      0.97      0.94      3851
           2       0.85      0.84      0.85       819

    accuracy                           0.90      4957
   macro avg       0.76      0.65      0.66      4957
weighted avg       0.88      0.90      0.88      4957

Overall AUC: 0.9183694395033682
Subgroup AUC: 0.9221678678494207
BPSN AUC: 0.9148649071003655
BNSP AUC: 0.9221678678494207
0.8481012658227848
              precision    recall  f1-score   support

           0       0.73      0.10      0.17        82
           1       0.84      0.99      0.91       471
           2       0.95      0.78      0.86        79

    accuracy                           0.85       632
   macro avg       0.84      0.62      0.65       632
weighted avg       0.84      0.85      0.81       632



In [6]:
# mitigate, re-weighting
overall_auc, subgroup_auc, bpsn_auc, bnsp_auc = correct_bias_subgroup_weights(df, 'clean_tweet', 'protected_attribute', multi_class=True)

Accuracy: 0.7924
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.05      0.09       287
           1       0.79      1.00      0.88      3851
           2       0.97      0.09      0.17       819

    accuracy                           0.79      4957
   macro avg       0.76      0.38      0.38      4957
weighted avg       0.80      0.79      0.72      4957

Overall AUC: 0.8280217645675009
Subgroup AUC: 0.8993353649121817
BPSN AUC: 0.8173775950907339
BNSP AUC: 0.8993353649121817


In [ ]:
# Adversarial debiasing - killed me dead, that it did
overall_auc, subgroup_auc, bpsn_auc, bnsp_auc = correct_bias_subgroup_ad(df, 'clean_tweet', 'protected_attribute', multi_class=True)

155/155 [==============================] - 0s 2ms/step
4957
4957
14871
Accuracy: 0.8727
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.06      0.11       287
           1       0.88      0.98      0.93      3851
           2       0.86      0.65      0.74       819

    accuracy                           0.87      4957
   macro avg       0.76      0.56      0.59      4957
weighted avg       0.85      0.87      0.85      4957

[0.04080876 0.84923935 0.10995195 ... 0.01230817 0.97723126 0.01046051]
ROC-AUC: 0.8880134999813404
Overall AUC: 0.8880134999813404
Subgroup AUC: 0.865586639571199
BPSN AUC: 0.8827878406326195
BNSP AUC: 0.865586639571199


In [17]:
metrics = pd.DataFrame()
row = {
    'subgroup': df['protected_attribute'].unique()[1:].copy(),
    'bspn_auc': 0.9221678678494207,
    'bpsn_auc': 0.8173775950907339,
    'subgroup_auc': 0.9221678678494207,
    'method': 'Original'
}
metrics = pd.concat([metrics, pd.DataFrame(row)], axis=0)

row = {
    'subgroup': df['protected_attribute'].unique()[1:].copy(),
    'bspn_auc': 0.8993353649121817,
    'bpsn_auc': 0.9148649071003655,
    'subgroup_auc': 0.8993353649121817,
    'method': 'Correct Bias Weights'
}
metrics = pd.concat([metrics, pd.DataFrame(row)], axis=0)

row = {
    'subgroup': df['protected_attribute'].unique()[1:].copy(),
    'bspn_auc': 0.865586639571199,
    'bpsn_auc': 0.8827878406326195,
    'subgroup_auc': 0.865586639571199,
    'method': 'Adversarial Debiasing'
}
metrics = pd.concat([metrics, pd.DataFrame(row)], axis=0)

In [25]:
metrics

,subgroup,bspn_auc,bpsn_auc,subgroup_auc,method
0,1,0.922168,0.817378,0.922168,Original
0,1,0.899335,0.914865,0.899335,Correct Bias Weights
0,1,0.865587,0.882788,0.865587,Adversarial Debiasing


In [19]:
# Saving to show in streamlit
metrics.to_csv('metrics.csv', index=False)